# Binary QSAR in Python - LabMol

Script version 2 - 11/02/2019

Developed by: Steven Hall


#  <font color='blue'> Model building with Morgan fingerprint and RF, SMV and GBM</font>

## Model building - Morgan_RF

In [1]:
# Importing packages 
from rdkit import Chem, DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef, roc_curve, precision_recall_curve, roc_auc_score
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score
import seaborn as sns
from pandas import DataFrame
import os

In [2]:
#Reading molecules and activity (0 and 1) from SDF
fname = "C:/Users/steve/Documents/Datasets/dermal/Dermal_Modeling_set.sdf"

mols = []
y = []
for mol in Chem.SDMolSupplier(fname):
    if mol is not None:
        mols.append(mol)
        y.append(mol.GetIntProp("Binary"))

ValueError: key `Binary` exists but does not result in an integer value

In [ ]:
# Calculate descriptors (fingerprints) and convert them into numpy array

# generate binary Morgan fingerprint with radius 2
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2,nBits=1024) for m in mols]

def rdkit_numpy_convert(fp):
    output = []
    for f in fp:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [ ]:
x = rdkit_numpy_convert(fp)

In [ ]:
# Check the number of compounds
len(x)

In [1]:
# check wether the data set is balanced
sum(y) / len(y)

NameError: name 'y' is not defined

In [11]:
#Set random seed to make all further calculations reproducible
seed = 42

In [12]:
# randomly select 20% of compounds as test set
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.20, random_state=seed)

In [13]:
#Create folds for cross-validation
cv = StratifiedKFold(n_splits=5, random_state=0)

In [14]:
# print out ids of folds
for i, (train_index, test_index) in enumerate(cv.split(x_tr, y_tr)):
    print("\nFold_" + str(i+1))
    print("TRAIN:", train_index)
    print("TEST:", test_index)


Fold_1
('TRAIN:', array([115, 117, 118, 119, 125, 126, 127, 128, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187,
       188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,
       201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
       214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252,
       253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265,
       266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278,
       279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291,
       292, 293, 294, 295, 296, 297, 298, 299

In [15]:
#Scale X
#This step may be crucial for certain modeling approaches lke SVM. 
#In the case of binary fingerprints it may be less useful.
# obtain scale object which can be further applied to scale any data to fit the training set
scale = StandardScaler().fit(x_tr)
x_tr = scale.transform(x_tr)


In [16]:
# it is a good idea to save it for future use


In [17]:
#Search for optimal tuning parameters and build the model
# create grid search dictionary
param_grid = {"max_features": [x_tr.shape[1] // 10, x_tr.shape[1] // 7, x_tr.shape[1] // 5, x_tr.shape[1] // 3], "n_estimators": [100, 250, 500]}

In [18]:
print(param_grid)

{'max_features': [102L, 146L, 204L, 341L], 'n_estimators': [100, 250, 500]}


In [19]:
# setup model building
rf = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [20]:
# run model building
rf.fit(x_tr, y_tr)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   39.4s
[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:   57.7s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=False),
       error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=2,
       param_grid={'max_features': [102L, 146L, 204L, 341L], 'n_estimators': [100, 250, 500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [21]:
rf.best_params_

{'max_features': 102L, 'n_estimators': 250}

In [22]:
rf.best_score_

0.7528641571194763

In [23]:
rf.cv_results_['mean_test_score']

array([0.74959083, 0.75286416, 0.7512275 , 0.74140753, 0.74140753,
       0.73977087, 0.73649755, 0.74468085, 0.73813421, 0.73649755,
       0.72667758, 0.7299509 ])

In [24]:
rf.cv_results_['params']

[{'max_features': 102L, 'n_estimators': 100},
 {'max_features': 102L, 'n_estimators': 250},
 {'max_features': 102L, 'n_estimators': 500},
 {'max_features': 146L, 'n_estimators': 100},
 {'max_features': 146L, 'n_estimators': 250},
 {'max_features': 146L, 'n_estimators': 500},
 {'max_features': 204L, 'n_estimators': 100},
 {'max_features': 204L, 'n_estimators': 250},
 {'max_features': 204L, 'n_estimators': 500},
 {'max_features': 341L, 'n_estimators': 100},
 {'max_features': 341L, 'n_estimators': 250},
 {'max_features': 341L, 'n_estimators': 500}]

In [25]:
#Save model - pkl file
tuple_objects = (rf, x_tr, y_tr)

joblib.dump(rf.best_estimator_, "C:/Users/steve/rf_clf.pkl")

['C:/Users/steve/rf_clf.pkl']

In [ ]:
#Predict test set compounds
# load scale if necessary
scale = joblib.load("")

In [ ]:
# scale descriptors of the test set compounds
x_ts = scale.transform(x_ts)

In [ ]:
# predict Outcome class
pred_rf = rf.predict(x_ts)

In [ ]:
pred_rf

#### Statistics - Morgan-RF

In [ ]:
#Calculate statistics for test set predictions

In [ ]:
# calc statistics
accuracy = accuracy_score(y_ts, pred_rf)
mcc = matthews_corrcoef(y_ts, pred_rf)
kappa = cohen_kappa_score(y_ts, pred_rf)
print("Accuracy = ", accuracy)
print("MCC = ", mcc)
print("Kappa = ", kappa)

In [ ]:
#DA estimation
# if the model includes several ones like RF models or consensus models (or for probabilistic models)
# we can calculate consistency of predictions amongs those models and use it for estimation of applicability domain
#just remove the hashtag to execute
pred_prob_rf = rf.predict_proba(x_ts)

In [ ]:
# probablity
pred_prob_rf

In [ ]:
# setup threshold
threshold = 0.8

In [ ]:
# calc maximum predicted probability for each row (compound) and compare to the threshold
da = np.amax(pred_prob_rf, axis=1) > threshold

In [ ]:
# calc coverage
coverage = sum(da) / len(da)

In [ ]:
# calc statistics (da)
accuracy_da = accuracy_score(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
# calc statistics (da)
mcc_da = matthews_corrcoef(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
# calc statistics (da)
kappa_da = cohen_kappa_score(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
# calc statistics (da)
coverage_da = sum(da) / len(da)

In [ ]:
# print statistics (da)
print("Accuracy_DA = ", accuracy_da)
print("MCC_DA = ", mcc_da)
print("Kappa_DA = ", kappa_da)
print("Coverage_DA = ", coverage_da)

In [ ]:
####################################################################################################
#ROC curve

# IMPORTANT: first argument is true values, second argument is predicted probabilities
# we pass y_test and y_pred_prob
# we do not use y_pred_class, because it will give incorrect results without generating an error
# roc_curve returns 3 objects fpr, tpr, thresholds
# fpr: false positive rate
# tpr: true positive rate

In [ ]:
#Store the predicted probabilities for class 1
y_pred_prob = rf.predict_proba(x_ts)[:, 1]

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_ts, y_pred_prob)

In [ ]:
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for malaria')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)

In [ ]:
#############################################################################################
#Model evaluation - confusion matrix

In [ ]:
# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_ts, pred_rf)

In [ ]:
print(confusion)

In [ ]:
#[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

In [ ]:
#Classification error or misclassification rate
classification_error = (FP + FN) / float(TP + TN + FP + FN)

In [ ]:
print(classification_error)
print(1 - metrics.accuracy_score(y_ts, pred_rf))

In [ ]:
#Sensitivity
sensitivity = TP / float(FN + TP)

In [ ]:
print(sensitivity)
print(metrics.recall_score(y_ts, pred_rf))

In [ ]:
#Specificity
specificity = TN / (TN + FP)
print(specificity)

In [ ]:
#False positive rate (alfa)
false_positive_rate = FP / float(TN + FP)

In [ ]:
print(false_positive_rate)
print(1 - (specificity))

In [ ]:
#False negative rate (beta)
false_negative_rate = FN / float(TP+FN)
print(false_negative_rate)
print(1 - (sensitivity))

In [ ]:
#Precision
precision = TP / float(TP + FP)

In [ ]:
print(precision)
print(metrics.precision_score(y_ts, pred_rf))

In [ ]:
#PPV
positive_pred_value = TP / float(TP + FP)
print(positive_pred_value)

In [ ]:
#NPV
negative_pred_value = TN / float(TN + FN)
print(negative_pred_value)

In [ ]:
#AUC
auc = metrics.roc_auc_score(y_ts, y_pred_prob)
print(auc)

In [ ]:
#Cross-validated AUC
cv_auc = cross_val_score(rf, x, y, cv=2, scoring='roc_auc').mean()

In [ ]:
####################################################################################################
# Print all metrics to report

In [ ]:
print("Classification error = ", classification_error)
print("Sensitivity = ", sensitivity)
print("Specificity = ", specificity)
print("False positive rate = ", false_positive_rate)
print("False negative rate = ", false_negative_rate)
print("Precision = ", precision)
print("PPV = ", positive_pred_value)
print("NPV = ", negative_pred_value)
print("AUC = ", metrics.roc_auc_score(y_ts, y_pred_prob))
print("AUC mean 5-fold = ", cv_auc)

In [ ]:
# Bar graph with metrics
sns.set_style("whitegrid")
stats = [accuracy, mcc, kappa, sensitivity, specificity, positive_pred_value, negative_pred_value, auc]
labels = ["Acc", "MCC", "Kappa", "Se", "Sp", "PPV", "NPV", "AUC"]
report = sns.barplot(x=labels, y=stats)
plt.savefig('report_rf_morgan.png', dpi=300)

In [ ]:
#converting calculated metrics into a pandas dataframe
metrics = DataFrame({'Accuracy': accuracy, 'MCC': mcc, 'Kappa': kappa, 
                     'Accuracy_DA': accuracy_da, 'MCC_DA': mcc_da, 'Kappa_da': kappa_da, 
                     "Classification error": classification_error, "Sensitivity": sensitivity,
                    "Specificity": specificity, "False positive rate": false_positive_rate,
                    "False negative rate": false_negative_rate, "Precision": precision, "PPV": positive_pred_value,
                    "NPV": negative_pred_value, 'AUC': auc, 'Cv_AUC': cv_auc, 'Coverage': coverage, 
                     'Coverage_DA': coverage_da}, index=[0])

In [ ]:
# Print the dataframe
print(metrics)

In [ ]:
# Saving the dataframe as excel file
metrics.to_excel("C:/Users/tiofi/Dropbox/Python/QSAR_python/Teofilo/metrics_Morgan_Rf.xlsx", sheet_name= "Sheet1")

## Model building - Morgan_SVM

In [ ]:
# create grid search dictionary
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [ ]:
# setup model building
svm = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, n_jobs=2, cv=cv, verbose=1)

In [ ]:
# run model building
svm.fit(x_tr, y_tr)

In [ ]:
svm.best_score_

In [ ]:
svm.best_params_

In [ ]:
# save model
joblib.dump(svm, "C:/Users/tiofi/Dropbox/Python/QSAR_python/Teofilo/malaria_python_svm_morgan.pkl", compress=3)

In [ ]:
# predict Outcome class
pred_svm = svm.predict(x_ts)

In [ ]:
pred_svm

#### Statistics - Morgan_SVM

In [ ]:
# calc statistics
accuracy = accuracy_score(y_ts, pred_svm)
mcc = matthews_corrcoef(y_ts, pred_svm)
kappa = cohen_kappa_score(y_ts, pred_svm)
print("Accuracy = ", accuracy)
print("MCC = ", mcc)
print("Kappa = ", kappa)

In [ ]:
# estimate applicability domain and calc stat
pred_prob = svm.predict_proba(x_ts)

In [ ]:
da = np.amax(pred_prob, axis=1) > threshold

In [ ]:
# calc coverage
coverage = sum(da) / len(da)

In [ ]:
# calc statistics (da)
accuracy_da = accuracy_score(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
# calc statistics (da)
mcc_da = matthews_corrcoef(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
# calc statistics (da)
kappa_da = cohen_kappa_score(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
# calc statistics (da)
coverage_da = sum(da) / len(da)

In [ ]:
# print statistics (da)
print("Accuracy_DA = ", accuracy_da)
print("MCC_DA = ", mcc_da)
print("Kappa_DA = ", kappa_da)
print("Coverage_DA = ", coverage_da)

In [ ]:
####################################################################################################
#ROC curve

In [ ]:
#Store the predicted probabilities for class 1
y_pred_prob = svm.predict_proba(x_ts)[:, 1]

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_ts, y_pred_prob)

In [ ]:
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for malaria')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)

In [ ]:
# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_ts, pred_svm)

In [ ]:
print(confusion)

In [ ]:
#[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

In [ ]:
# Classification accuracy - use float to perform true division, not integer division
print((TP + TN) / float(TP + TN + FP + FN))
print(metrics.accuracy_score(y_ts, pred_svm))

In [ ]:
#Classification error or misclassification rate
classification_error = (FP + FN) / float(TP + TN + FP + FN)

In [ ]:
print(classification_error)
print(1 - metrics.accuracy_score(y_ts, pred_svm))

In [ ]:
#Sensitivity
sensitivity = TP / float(FN + TP)

In [ ]:
print(sensitivity)
print(metrics.recall_score(y_ts, pred_svm))

In [ ]:
#Specificity
specificity = TN / (TN + FP)
print(specificity)

In [ ]:
#False positive rate (alfa)
false_positive_rate = FP / float(TN + FP)

In [ ]:
print(false_positive_rate)
print(1 - (specificity))

In [ ]:
#False negative rate (beta)
false_negative_rate = FN / float(TP+FN)
print(false_negative_rate)
print(1 - (sensitivity))

In [ ]:
#Precision
precision = TP / float(TP + FP)

In [ ]:
print(precision)
print(metrics.precision_score(y_ts, pred_svm))

In [ ]:
#PPV
positive_pred_value = TP / float(TP + FP)
print(positive_pred_value)

In [ ]:
#NPV
negative_pred_value = TN / float(TN + FN)
print(negative_pred_value)

In [ ]:
#AUC
auc = metrics.roc_auc_score(y_ts, y_pred_prob)
print(auc)

In [ ]:
# Cross-validated AUC
cv_auc = cross_val_score(svm, x, y, cv=2, scoring='roc_auc').mean()

In [ ]:
####################################################################################################
# Print all metrics to report

In [ ]:
print("Classification error = ", classification_error)
print("Sensitivity = ", sensitivity)
print("Specificity = ", specificity)
print("False positive rate = ", false_positive_rate)
print("False negative rate = ", false_negative_rate)
print("Precision = ", precision)
print("PPV = ", positive_pred_value)
print("NPV = ", negative_pred_value)
print("AUC = ", metrics.roc_auc_score(y_ts, y_pred_prob))
print("AUC mean 5-fold = ", cv_auc)

In [ ]:
# Bar graph with metrics
sns.set_style("whitegrid")
stats = [accuracy, mcc, kappa, sensitivity, specificity, positive_pred_value, negative_pred_value, auc]
labels = ["Acc", "MCC", "Kappa", "Se", "Sp", "PPV", "NPV", "AUC"]
report = sns.barplot(x=labels, y=stats)
plt.savefig('report_svm_morgan.png', dpi=300)

In [ ]:
#converting calculated metrics into a pandas dataframe
metrics = DataFrame({'Accuracy': accuracy, 'MCC': mcc, 'Kappa': kappa,
                     'Accuracy_DA': accuracy_da, 'MCC_DA': mcc_da, 'Kappa_da': kappa_da,
                     "Classification error": classification_error, "Sensitivity": sensitivity,
                    "Specificity": specificity, "False positive rate": false_positive_rate,
                    "False negative rate": false_negative_rate, "Precision": precision, "PPV": positive_pred_value,
                    "NPV": negative_pred_value, 'AUC': auc, 'Cv_AUC': cv_auc, 'Coverage': coverage,
                     'Coverage_DA': coverage_da}, index=[0])

In [ ]:
# Print the dataframe
print(metrics)

In [ ]:
# Saving the dataframe as excel file
metrics.to_excel("C:/Users/tiofi/Dropbox/Python/QSAR_python/Teofilo/metrics_Morgan_SVM.xlsx", sheet_name= "Sheet1")

## Model building - Morgan_GBM

In [ ]:
param_grid = {"n_estimators": [100, 200, 300, 400, 500]}
gbm = GridSearchCV(GradientBoostingClassifier(subsample=0.5, max_features=0.5), 
                   param_grid, n_jobs=2, cv=cv, verbose=1)

In [ ]:
# run model building
gbm.fit(x_tr, y_tr)

In [ ]:
gbm.best_score_

In [ ]:
gbm.best_params_

In [ ]:
pred_gbm = gbm.predict(x_ts)

In [ ]:
#Save model - pkl file
joblib.dump(gbm, "C:/Users/tiofi/Dropbox/Python/QSAR_python/Teofilo/malaria_python_gbm_morgan.pkl", compress=3)

#### Statistics - Morgan_GBM

In [ ]:
# calc statistics
accuracy = accuracy_score(y_ts, pred_gbm)
mcc = matthews_corrcoef(y_ts, pred_gbm)
kappa = cohen_kappa_score(y_ts, pred_gbm)
print("Accuracy = ", accuracy)
print("MCC = ", mcc)
print("Kappa = ", kappa)

In [ ]:
# estimate applicability domain and calc stat
pred_prob = gbm.predict_proba(x_ts)

In [ ]:
da = np.amax(pred_prob, axis=1) > threshold

In [ ]:
# calc coverage
coverage = sum(da) / len(da)

In [ ]:
# calc statistics (da)
accuracy_da = accuracy_score(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
# calc statistics (da)
mcc_da = matthews_corrcoef(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
# calc statistics (da)
kappa_da = cohen_kappa_score(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
# calc statistics (da)
coverage_da = sum(da) / len(da)

In [ ]:
# print statistics (da)
print("Accuracy_DA = ", accuracy_da)
print("MCC_DA = ", mcc_da)
print("Kappa_DA = ", kappa_da)
print("Coverage_DA = ", coverage_da)

In [ ]:
####################################################################################################
#ROC curve

In [ ]:
#Store the predicted probabilities for class 1
y_pred_prob = gbm.predict_proba(x_ts)[:, 1]

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_ts, y_pred_prob)

In [ ]:
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for malaria')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)

In [ ]:
# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_ts, pred_gbm)

In [ ]:
print(confusion)

In [ ]:
#[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

In [ ]:
# Classification accuracy - use float to perform true division, not integer division
print((TP + TN) / float(TP + TN + FP + FN))
print(metrics.accuracy_score(y_ts, pred_gbm))

In [ ]:
#Classification error or misclassification rate
classification_error = (FP + FN) / float(TP + TN + FP + FN)

In [ ]:
print(classification_error)
print(1 - metrics.accuracy_score(y_ts, pred_gbm))

In [ ]:
#Sensitivity
sensitivity = TP / float(FN + TP)

In [ ]:
print(sensitivity)
print(metrics.recall_score(y_ts, pred_gbm))

In [ ]:
#Specificity
specificity = TN / (TN + FP)
print(specificity)

In [ ]:
#False positive rate (alfa)
false_positive_rate = FP / float(TN + FP)

In [ ]:
print(false_positive_rate)
print(1 - (specificity))

In [ ]:
#False negative rate (beta)
false_negative_rate = FN / float(TP+FN)
print(false_negative_rate)
print(1 - (sensitivity))

In [ ]:
#Precision
precision = TP / float(TP + FP)

In [ ]:
print(precision)
print(metrics.precision_score(y_ts, pred_gbm))

In [ ]:
#PPV
positive_pred_value = TP / float(TP + FP)
print(positive_pred_value)

In [ ]:
#NPV
negative_pred_value = TN / float(TN + FN)
print(negative_pred_value)

In [ ]:
#AUC
auc = metrics.roc_auc_score(y_ts, y_pred_prob)
print(auc)

In [ ]:
# Cross-validated AUC
cv_auc = cross_val_score(gbm, x, y, cv=2, scoring='roc_auc').mean()

In [ ]:
####################################################################################################
# Print all metrics to report

In [ ]:
print("Classification error = ", classification_error)
print("Sensitivity = ", sensitivity)
print("Specificity = ", specificity)
print("False positive rate = ", false_positive_rate)
print("False negative rate = ", false_negative_rate)
print("Precision = ", precision)
print("PPV = ", positive_pred_value)
print("NPV = ", negative_pred_value)
print("AUC = ", metrics.roc_auc_score(y_ts, y_pred_prob))
print("AUC mean 5-fold = ", cv_auc)

In [ ]:
# Bar graph with metrics
sns.set_style("whitegrid")
stats = [accuracy, mcc, kappa, sensitivity, specificity, positive_pred_value, negative_pred_value, auc]
labels = ["Acc", "MCC", "Kappa", "Se", "Sp", "PPV", "NPV", "AUC"]
report = sns.barplot(x=labels, y=stats)
plt.savefig('report_gbm_morgan.png', dpi=300)

In [ ]:
#converting calculated metrics into a pandas dataframe
metrics = DataFrame({'Accuracy': accuracy, 'MCC': mcc, 'Kappa': kappa,
                     'Accuracy_DA': accuracy_da, 'MCC_DA': mcc_da, 'Kappa_da': kappa_da,
                     "Classification error": classification_error, "Sensitivity": sensitivity,
                    "Specificity": specificity, "False positive rate": false_positive_rate,
                    "False negative rate": false_negative_rate, "Precision": precision, "PPV": positive_pred_value,
                    "NPV": negative_pred_value, 'AUC': auc, 'Cv_AUC': cv_auc, 'Coverage': coverage,
                     'Coverage_DA': coverage_da}, index=[0])

In [ ]:
# Print the dataframe
print(metrics)

In [ ]:
# Saving the dataframe as excel file
metrics.to_excel("C:/Users/tiofi/Dropbox/Python/QSAR_python/Teofilo/metrics_Morgan_GBM.xlsx", sheet_name= "Sheet1")

## Predict new molecules (virtual screening)

In [ ]:
# load molecules
kaira = "C:/Users/tiofi/Dropbox/Python/QSAR_python/Teofilo/PvCK1_XPbox15select-12a-8.sdf"

In [ ]:
mols = []
y = []
for mol in Chem.SDMolSupplier(kaira):
    if mol is not None:
        mols.append(mol)

In [ ]:
# generate binary Morgan fingerprint with radius 2
fp1 = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]

In [ ]:
def rdkit_numpy_convert(fp1):
    output = []
    for f in fp1:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [ ]:
x1 = rdkit_numpy_convert(fp1)

In [ ]:
#print(x.shape)
print(x1.shape)

In [ ]:
#Predict new data
m = joblib.load('C:/Users/tiofi/Dropbox/Python/QSAR_python/Teofilo/malaria_python_rf_morgan.pkl')
orig_pp = m.predict_proba(x1)
orig_pp

In [ ]:
# Verify the number of predicted compounds
print(orig_pp.shape)

Save the results in excel file

In [ ]:
# convert numpy array to pandas dataframe
vs = DataFrame(orig_pp, columns=['Prob_inactive', 'Prob_active'])

In [ ]:
# Save the pandas dataframe in excel file
vs.to_excel("C:/Users/tiofi/Dropbox/Python/QSAR_python/Teofilo/teste_results.xlsx", sheet_name= "Sheet1")

Save the results in excel file - method 2

In [ ]:
#################################################################################
import xlsxwriter

In [ ]:
workbook = xlsxwriter.Workbook('C:/Users/tiofi/Dropbox/Python/QSAR_python/Teofilo/pvck1.xlsx')
worksheet = workbook.add_worksheet()

In [ ]:
# Add a bold format to use to highlight cells.
bold = workbook.add_format({'bold': True})

In [ ]:
worksheet.set_column('A:C', 20)
worksheet.write('A1', 'Prob_active', bold)
worksheet.write('B1', 'Prob_inactive', bold)
worksheet.write('C1', 'Final consensus', bold)

In [ ]:
#Start from the first cell below the headers
row = 1
col = 0

In [ ]:
#Iterate over the data and write it out row by row
for prob_active, data in (orig_pp):
    worksheet.write(row, col,  data)
    worksheet.write(row, col + 1, 1-data)
    worksheet.write_array_formula('C2:C2', '{IF(A2:A2 >= 0.8, "Virtual hit", "Inactive")}') #falta melhorar isso aqui - contar colunas automaticamente
    row += 1                                                                               #escrever os smiles correspondentes tb, ou a estrutura 2D

In [ ]:
workbook.close()